# SnapShot preprocessing:
In this notebook, we process the snapshots to reconstruct the travel diaries.

#### adding required packages

In [1]:
import pandas as pd
from scipy.stats import gaussian_kde
import time
import numpy as np
import pickle
import concurrent.futures
from loky import ProcessPoolExecutor
from threading import Thread
import logging
import multiprocessing as mp
from multiprocess import Pool
from joblib import Parallel, delayed

### specifying the saving location 


In [10]:
savingLoc = "Y:/ZahraEftekhar/phase4/"

First, we have to cluster the consecative records that have the same location (i.e., the TAZ).

In [9]:
def cluster_records(interval,address):
    t1= time.time()
    clusterData = {}
    with open("{a}completePLUdata_{b}sec_dict.pickle".format(a=address,b= interval),'rb') as handle:
        snapDataSplit = pickle.load(handle)
    IDs = list(snapDataSplit.keys())
    for i,ID in enumerate(IDs):
        person = snapDataSplit[ID]
        person = person.reset_index(drop=False, inplace=False)
        newD = pd.DataFrame()
        for k,v in person.groupby((person.mzr_id.shift() != person.mzr_id).cumsum()):
            newD = newD.append(v.iloc[0,:])
        d = np.array(newD["TIME"].shift(periods=-1, fill_value=v.iloc[-1,:]["TIME"])) - np.array(newD["TIME"])
        d = [pd.Timedelta(d[i]).total_seconds() for i in np.arange(len(d))]
        newD["duration"] = d
        newD.columns = ['start', 'id', 'x', 'y', 'mzr_id', 'duration']
        clusterData[ID] = newD
    print(interval, ":  ",(time.time()-t1)//60,"minutes")
    with open('{a}clusterData_{b}sec.pickle'.format(a= address,b = interval), 'wb') as handle:
        pickle.dump(clusterData, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [14]:
#with this type of parallel processing execution of this cell takes only about 30 minutes for all the intervals! 
# if not using parallel processing it would be more than 2 hours!
intervals = [60,300,600,900,1200,1500,1800,2100,2400,2700,3000,3300,3600,4500,5400,6300,7200]
t2 = time.time()
results = Parallel(n_jobs=6)(delayed(cluster_records)(interval,savingLoc) for interval in intervals)
print(time.time()-t2)   

637.1613354682922
